In [1]:
!pip -q install timm==1.0.9 transformers==4.44.2 accelerate==0.34.2 --upgrade
!pip -q install openai-whisper==20231117 transformers==4.44.2 timm==1.0.9 accelerate==0.34.2 soundfile==0.12.1 --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 99.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

# Processed manifest from your preprocessing notebook
PROC_DIR = Path("/kaggle/input/thesis-data-processed")
MANIFEST = PROC_DIR / "hatemm_train_manifest.csv"  # created previously
assert MANIFEST.exists(), "Missing hatemm_train_manifest.csv. Run the preprocessing notebook first."

# Use the existing cache_hatemm layout you already created
CACHE_DIR = Path("/kaggle/input/thesis-data-processed")
V_FEAT_DIR = CACHE_DIR / "vision"   # existing .npy features (vision-generic)
A_FEAT_DIR = CACHE_DIR / "audio"    # existing .npy features (audio-generic)

print("Manifest:", MANIFEST)
print("Vision dir exists:", V_FEAT_DIR.exists(), V_FEAT_DIR)
print("Audio dir exists:", A_FEAT_DIR.exists(), A_FEAT_DIR)

df = pd.read_csv(MANIFEST)
print("Loaded manifest rows:", len(df))
df.head(2)


Manifest: /kaggle/input/thesis-data-processed/hatemm_train_manifest.csv
Vision dir exists: False /kaggle/input/thesis-data-processed/vision
Audio dir exists: False /kaggle/input/thesis-data-processed/audio
Loaded manifest rows: 1083


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,dataset,video_id,video_path,start_sec,end_sec,target_group,label,binary_offensive,sample_id,video_seconds,segment_seconds,is_broken,vision_feat_path,audio_feat_path,has_vision,has_audio
0,HateMM,hate_video_1.mp4,/kaggle/input/thesis-dataset/Thesis_dataset/ha...,NaN,NaN,['Blacks'],hateful,1,HateMM__hate_video_1.mp4__0_end__hateful,94.998,NaN,False,/kaggle/working/cache_hatemm/vision/HateMM__ha...,/kaggle/working/cache_hatemm/audio/HateMM__hat...,True,True
1,HateMM,hate_video_2.mp4,/kaggle/input/thesis-dataset/Thesis_dataset/ha...,NaN,NaN,['Blacks'],hateful,1,HateMM__hate_video_2.mp4__0_end__hateful,129.160,NaN,False,/kaggle/working/cache_hatemm/vision/HateMM__ha...,/kaggle/working/cache_hatemm/audio/HateMM__hat...,True,True


In [3]:
# === Rewrite feature paths in manifest to use a portable CACHE_DIR (INPUT is read-only) ===
from pathlib import Path
import pandas as pd
import numpy as np

# ---- CONFIG (chỉnh 2 dòng này) ----
PROC_DIR    = Path("/kaggle/input/thesis-data-processed")               # nơi bạn có quyền ghi
IN_MANIFEST = PROC_DIR / "hatemm_train_manifest.csv"          # manifest cũ do preprocessing tạo
CACHE_DIR   = Path("/kaggle/input/thesis-data-processed/features")     # <-- ĐỔI: tên folder bạn đã upload (INPUT - READ ONLY)

# Chỉ đọc từ INPUT; KHÔNG mkdir trong INPUT
V_FEAT_DIR = CACHE_DIR / "vision"
A_FEAT_DIR = CACHE_DIR / "audio"

assert IN_MANIFEST.exists(), f"Manifest not found: {IN_MANIFEST}"
df = pd.read_csv(IN_MANIFEST)

def ensure_col(df, col):
    if col not in df.columns:
        df[col] = pd.Series([np.nan]*len(df))
    return df

df = ensure_col(df, "sample_id")
df = ensure_col(df, "vision_feat_path")
df = ensure_col(df, "audio_feat_path")

def rewrite_to_cache_dir(path_value, sample_id: str, subdir: Path):
    """
    - Nếu path cũ có sẵn -> lấy basename và ghép lại với CACHE_DIR/<subdir>/<basename>.
    - Nếu path rỗng -> synthesize từ sample_id: CACHE_DIR/<subdir>/<sample_id>.npy
    """
    try:
        if isinstance(path_value, str) and path_value.strip():
            base = Path(path_value).name
            return str(subdir / base)
    except Exception:
        pass
    if isinstance(sample_id, str) and sample_id:
        return str(subdir / f"{sample_id}.npy")
    return np.nan

df["vision_feat_path"] = [
    rewrite_to_cache_dir(p, sid, V_FEAT_DIR) for p, sid in zip(df["vision_feat_path"], df["sample_id"])
]
df["audio_feat_path"] = [
    rewrite_to_cache_dir(p, sid, A_FEAT_DIR) for p, sid in zip(df["audio_feat_path"], df["sample_id"])
]

# (Optional) Kiểm tra tồn tại file trên INPUT
def exists_safe(p):
    try:
        return Path(p).exists()
    except Exception:
        return False

n_v_exist = int(df["vision_feat_path"].apply(exists_safe).sum())
n_a_exist = int(df["audio_feat_path"].apply(exists_safe).sum())
print(f"Vision features found: {n_v_exist}/{len(df)}")
print(f"Audio  features found: {n_a_exist}/{len(df)}")

# ---- SAVE ra /kaggle/working (có quyền ghi) ----
OUT_PROC_DIR = Path("/kaggle/working")
OUT_MANIFEST = OUT_PROC_DIR / "hatemm_train_manifest.rewritten.csv"
PROC_DIR.mkdir(parents=True, exist_ok=True)   # working dir -> OK
df.to_csv(OUT_MANIFEST, index=False)
print("Rewritten manifest saved to:", OUT_MANIFEST)

# Preview
df[["sample_id","label","vision_feat_path","audio_feat_path"]].head(5)

Vision features found: 1081/1083
Audio  features found: 1068/1083
Rewritten manifest saved to: /kaggle/working/hatemm_train_manifest.rewritten.csv


,sample_id,label,vision_feat_path,audio_feat_path
0,HateMM__hate_video_1.mp4__0_end__hateful,hateful,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
1,HateMM__hate_video_2.mp4__0_end__hateful,hateful,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
2,HateMM__non_hate_video_1.mp4__0_end__normal,normal,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
3,HateMM__hate_video_3.mp4__0_end__hateful,hateful,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
4,HateMM__non_hate_video_2.mp4__0_end__normal,normal,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...


In [4]:
from pathlib import Path
import pandas as pd

MANIFEST = Path("/kaggle/working/hatemm_train_manifest.rewritten.csv")
df = pd.read_csv(MANIFEST)

In [5]:
from pathlib import Path
import numpy as np

def safe_path(p):
    return str(p) if (isinstance(p, (str, Path)) and Path(p).exists()) else None

# If your manifest already has paths, trust them; otherwise, try to infer from sample_id
if "vision_feat_path" not in df.columns or "audio_feat_path" not in df.columns:
    # Infer .npy path by sample_id
    def guess_feat(sample_id, root: Path):
        cand = root / f"{sample_id}.npy"
        return str(cand) if cand.exists() else None

    df["vision_feat_path"] = df["sample_id"].apply(lambda sid: guess_feat(sid, V_FEAT_DIR))
    df["audio_feat_path"]  = df["sample_id"].apply(lambda sid: guess_feat(sid, A_FEAT_DIR))
else:
    # Normalize (if stored as relative paths)
    df["vision_feat_path"] = df["vision_feat_path"].apply(lambda p: safe_path(p))
    df["audio_feat_path"]  = df["audio_feat_path"].apply(lambda p: safe_path(p))

# Keep rows that have at least one modality available
usable = df[(df["vision_feat_path"].notna()) | (df["audio_feat_path"].notna())].copy()
print("Usable rows:", len(usable), "/", len(df))

# Label mapping
labels_sorted = sorted(usable["label"].unique())
label2id = {lbl:i for i,lbl in enumerate(labels_sorted)}
id2label = {i:lbl for lbl,i in label2id.items()}
print("Classes:", label2id)

Usable rows: 1083 / 1083
Classes: {'hateful': 0, 'normal': 1}


In [6]:
# === Replace Cell A2 with this robust builder ===
from pathlib import Path
import numpy as np
import pandas as pd

def load_npy_dim(path):
    try:
        p = Path(path)
        if p.exists():
            arr = np.load(p, mmap_mode="r")
            if arr.ndim == 1:
                return int(arr.shape[0])
            # if accidentally saved as (1,D) etc.
            return int(np.prod(arr.shape))
    except Exception:
        return np.nan
    return np.nan

def load_npy(path):
    try:
        p = Path(path)
        if p.exists():
            a = np.load(p)
            return a.squeeze().astype(np.float32)
    except Exception:
        return None
    return None

# --- Diagnose feature dimensions per modality ---
usable = df.copy()

usable["v_dim"] = usable["vision_feat_path"].apply(load_npy_dim) if "vision_feat_path" in usable.columns else np.nan
usable["a_dim"] = usable["audio_feat_path"].apply(load_npy_dim)  if "audio_feat_path"  in usable.columns else np.nan

# Count distribution
v_counts = usable["v_dim"].dropna().astype(int).value_counts().sort_index()
a_counts = usable["a_dim"].dropna().astype(int).value_counts().sort_index()
print("Vision dim counts:\n", v_counts.to_string())
print("\nAudio  dim counts:\n", a_counts.to_string())

# --- Strategy toggle ---
USE_PADDING = False   # False = keep only majority-dim; True = zero-pad to max dim

# Majority dims
v_major = int(v_counts.idxmax()) if len(v_counts)>0 else None
a_major = int(a_counts.idxmax()) if len(a_counts)>0 else None
print(f"\nChosen majority dims -> vision: {v_major}, audio: {a_major} (set USE_PADDING=True to pad instead)")

def to_dense_pad(vectors, target_dim=None):
    """Pad 1D vectors to same length (max or provided target_dim)."""
    if len(vectors) == 0:
        return np.zeros((0, 0), dtype=np.float32)
    if target_dim is None:
        target_dim = max(v.shape[0] for v in vectors)
    M = np.zeros((len(vectors), target_dim), dtype=np.float32)
    for i, v in enumerate(vectors):
        d = min(target_dim, v.shape[0])
        M[i, :d] = v[:d]
    return M

# --- Builders for three setups ---
labels_sorted = sorted(usable["label"].unique())
label2id = {lbl:i for i,lbl in enumerate(labels_sorted)}
id2label = {i:lbl for lbl,i in label2id.items()}
print("Classes:", label2id)

def build_V_only(df_in):
    rows = []
    for _, r in df_in.iterrows():
        p = r.get("vision_feat_path")
        v = load_npy(p) if isinstance(p, str) else None
        if v is None:
            continue
        rows.append((v, label2id[r["label"]], str(r["video_id"])))
    if not rows:
        return np.zeros((0,0), np.float32), np.zeros((0,), np.int64), np.array([])
    vecs, Y, G = zip(*rows)
    if USE_PADDING:
        X = to_dense_pad(list(vecs))  # pad to max vision dim
    else:
        # keep only majority-dim vectors
        keep = [v for v in vecs if v.shape[0] == v_major]
        idxs = [i for i,v in enumerate(vecs) if v.shape[0] == v_major]
        X = np.stack(keep, axis=0)
        Y = np.array([Y[i] for i in idxs], dtype=np.int64)
        G = np.array([G[i] for i in idxs])
    return X, Y, G

def build_A_only(df_in):
    rows = []
    for _, r in df_in.iterrows():
        p = r.get("audio_feat_path")
        a = load_npy(p) if isinstance(p, str) else None
        if a is None:
            continue
        rows.append((a, label2id[r["label"]], str(r["video_id"])))
    if not rows:
        return np.zeros((0,0), np.float32), np.zeros((0,), np.int64), np.array([])
    vecs, Y, G = zip(*rows)
    if USE_PADDING:
        X = to_dense_pad(list(vecs))  # pad to max audio dim
    else:
        keep = [a for a in vecs if a.shape[0] == a_major]
        idxs = [i for i,a in enumerate(vecs) if a.shape[0] == a_major]
        X = np.stack(keep, axis=0)
        Y = np.array([Y[i] for i in idxs], dtype=np.int64)
        G = np.array([G[i] for i in idxs])
    return X, Y, G

def build_VA_fusion(df_in):
    rows = []
    for _, r in df_in.iterrows():
        pv = r.get("vision_feat_path"); pa = r.get("audio_feat_path")
        v = load_npy(pv) if isinstance(pv, str) else None
        a = load_npy(pa) if isinstance(pa, str) else None
        if v is None or a is None:
            continue
        rows.append((v, a, label2id[r["label"]], str(r["video_id"])))
    if not rows:
        return np.zeros((0,0), np.float32), np.zeros((0,), np.int64), np.array([])
    V, A, Y, G = zip(*rows)

    if USE_PADDING:
        Vd = to_dense_pad(list(V))   # pad each modality separately
        Ad = to_dense_pad(list(A))
        X  = np.concatenate([Vd, Ad], axis=1)
    else:
        # keep only samples where dims == majority for BOTH modalities
        idxs = [i for i,(v,a) in enumerate(zip(V,A)) if v.shape[0]==v_major and a.shape[0]==a_major]
        if len(idxs)==0:
            raise RuntimeError("No overlapping majority-dim samples for fusion. Consider USE_PADDING=True.")
        V_keep = np.stack([V[i] for i in idxs], axis=0)
        A_keep = np.stack([A[i] for i in idxs], axis=0)
        X = np.concatenate([V_keep, A_keep], axis=1)
        Y = np.array([Y[i] for i in idxs], dtype=np.int64)
        G = np.array([G[i] for i in idxs])
    return X, Y, G

# --- Build matrices ---
Xv,  Yv,  Gv  = build_V_only(usable)
Xa,  Ya,  Ga  = build_A_only(usable)
Xva, Yva, Gva = build_VA_fusion(usable)

print("Vision-only:", Xv.shape, Yv.shape)
print("Audio-only :", Xa.shape, Ya.shape)
print("Fusion     :", Xva.shape, Yva.shape)

# Little sanity check: non-empty?
if Xv.size == 0 and Xa.size == 0:
    print("WARNING: No usable features found. Check feature folders & sample_id naming.")


Vision dim counts:
 v_dim
2048    1081

Audio  dim counts:
 a_dim
64    1068

Chosen majority dims -> vision: 2048, audio: 64 (set USE_PADDING=True to pad instead)
Classes: {'hateful': 0, 'normal': 1}
Vision-only: (1081, 2048) (1081,)
Audio-only : (1068, 64) (1068,)
Fusion     : (1066, 2112) (1066,)


In [7]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np
import pandas as pd

def run_groupkfold(X, Y, groups, name=""):
    gkf = GroupKFold(n_splits=min(5, len(np.unique(groups))))
    def go(clf):
        yt, yp = [], []
        for tr, te in gkf.split(X, Y, groups):
            clf.fit(X[tr], Y[tr])
            yp.append(clf.predict(X[te])); yt.append(Y[te])
        yt = np.concatenate(yt); yp = np.concatenate(yp)
        acc = accuracy_score(yt, yp)
        f1m = f1_score(yt, yp, average="macro")
        rep = classification_report(yt, yp, target_names=[k for k,_ in sorted(label2id.items(), key=lambda kv: kv[1])], zero_division=0)
        return acc, f1m, rep

    lr  = LogisticRegression(penalty="l2", C=1.0, solver="saga", class_weight="balanced", max_iter=3000, n_jobs=-1)
    mlp = MLPClassifier(hidden_layer_sizes=(256,), activation="relu", alpha=1e-4,
                        batch_size=128, learning_rate_init=1e-3, max_iter=80, early_stopping=True, random_state=42)

    acc_lr, f1_lr, rep_lr = go(lr)
    acc_mlp, f1_mlp, rep_mlp = go(mlp)

    print(f"[{name}] LR  acc={acc_lr:.3f}  f1m={f1_lr:.3f}")
    print(f"[{name}] MLP acc={acc_mlp:.3f} f1m={f1_mlp:.3f}")
    return {
        "name": name,
        "LR_acc": acc_lr, "LR_f1m": f1_lr, "report_LR": rep_lr,
        "MLP_acc": acc_mlp, "MLP_f1m": f1_mlp, "report_MLP": rep_mlp
    }

results = []
results.append(run_groupkfold(Xv,  Yv,  Gv,  "Vision-only (generic)"))
results.append(run_groupkfold(Xa,  Ya,  Ga,  "Audio-only (generic)"))
results.append(run_groupkfold(Xva, Yva, Gva, "Early Fusion (V+A generic)"))

summary = pd.DataFrame([{k:v for k,v in r.items() if isinstance(v,(int,float,str))} for r in results]).sort_values("MLP_f1m", ascending=False)
summary_path = OUT_PROC_DIR / "baseline_summary_generic.csv"
summary.to_csv(summary_path, index=False)
print("Saved:", summary_path)
summary


[Vision-only (generic)] LR  acc=0.716  f1m=0.705
[Vision-only (generic)] MLP acc=0.717 f1m=0.702


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[Audio-only (generic)] LR  acc=0.632  f1m=0.626
[Audio-only (generic)] MLP acc=0.576 f1m=0.467


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[Early Fusion (V+A generic)] LR  acc=0.735  f1m=0.726
[Early Fusion (V+A generic)] MLP acc=0.707 f1m=0.681
Saved: /kaggle/working/baseline_summary_generic.csv


,name,LR_acc,LR_f1m,report_LR,MLP_acc,MLP_f1m,report_MLP
0,Vision-only (generic),0.716004,0.705085,precision recall f1-score ...,0.716929,0.702123,precision recall f1-score ...
2,Early Fusion (V+A generic),0.734522,0.726460,precision recall f1-score ...,0.707317,0.681407,precision recall f1-score ...
1,Audio-only (generic),0.632022,0.625866,precision recall f1-score ...,0.575843,0.466786,precision recall f1-score ...


In [8]:
# Nothing heavy to install; sklearn is preinstalled on Kaggle.
import sklearn, sys, numpy, pandas
print("sklearn:", sklearn.__version__)

sklearn: 1.2.2


In [9]:
from pathlib import Path
import pandas as pd
import numpy as np

# === Paths ===
PROC_DIR    = Path("/kaggle/working")                # where you saved manifests
MANIFEST    = PROC_DIR / "hatemm_train_manifest.rewritten.csv" # from the rewrite step earlier
assert MANIFEST.exists(), f"Missing manifest: {MANIFEST}"

# This is the INPUT folder you uploaded that contains:
#   /vision/*.npy  and  /audio/*.npy
CACHE_DIR = Path("/kaggle/input/thesis-data-processed/features")        # <-- CHANGE to your uploaded dataset folder name
V_DIR     = CACHE_DIR / "vision"
A_DIR     = CACHE_DIR / "audio"
print("Manifest:", MANIFEST)
print("Vision dir exists:", V_DIR.exists(), V_DIR)
print("Audio  dir exists:", A_DIR.exists(), A_DIR)

df = pd.read_csv(MANIFEST)
print("Loaded rows:", len(df))
df.head(3)

Manifest: /kaggle/working/hatemm_train_manifest.rewritten.csv
Vision dir exists: True /kaggle/input/thesis-data-processed/features/vision
Audio  dir exists: True /kaggle/input/thesis-data-processed/features/audio
Loaded rows: 1083


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,dataset,video_id,video_path,start_sec,end_sec,target_group,label,binary_offensive,sample_id,video_seconds,segment_seconds,is_broken,vision_feat_path,audio_feat_path,has_vision,has_audio
0,HateMM,hate_video_1.mp4,/kaggle/input/thesis-dataset/Thesis_dataset/ha...,NaN,NaN,['Blacks'],hateful,1,HateMM__hate_video_1.mp4__0_end__hateful,94.998,NaN,False,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...,True,True
1,HateMM,hate_video_2.mp4,/kaggle/input/thesis-dataset/Thesis_dataset/ha...,NaN,NaN,['Blacks'],hateful,1,HateMM__hate_video_2.mp4__0_end__hateful,129.160,NaN,False,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...,True,True
2,HateMM,non_hate_video_1.mp4,/kaggle/input/thesis-dataset/Thesis_dataset/no...,NaN,NaN,['Others'],normal,0,HateMM__non_hate_video_1.mp4__0_end__normal,108.832,NaN,False,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...,True,True


In [10]:
from pathlib import Path
import numpy as np

def fix_col_path(series, target_root):
    out = []
    for p in series.fillna(""):
        base = Path(p).name if isinstance(p, str) and p else None
        out.append(str(target_root / base) if base else np.nan)
    return out

# Ensure columns exist and rewrite to current CACHE_DIR
if "vision_feat_path" not in df.columns: df["vision_feat_path"] = np.nan
if "audio_feat_path"  not in df.columns: df["audio_feat_path"]  = np.nan
df["vision_feat_path"] = fix_col_path(df["vision_feat_path"], V_DIR)
df["audio_feat_path"]  = fix_col_path(df["audio_feat_path"],  A_DIR)

# Keep only rows that have at least one modality (we’ll build 3 matrices separately)
usable = df[(df["vision_feat_path"].notna()) | (df["audio_feat_path"].notna())].copy()
print("Usable rows:", len(usable), "/", len(df))

# Label map
labels_sorted = sorted(usable["label"].unique())
label2id = {lbl:i for i,lbl in enumerate(labels_sorted)}
id2label = {i:lbl for lbl,i in label2id.items()}
print("Classes:", label2id)

Usable rows: 1083 / 1083
Classes: {'hateful': 0, 'normal': 1}


In [11]:
# === TV3 (fixed) — Robust builders for V, A, and V+A ===
import numpy as np
from pathlib import Path

def load_vec(p):
    try:
        p = Path(p)
        if p.exists():
            a = np.load(p, mmap_mode=None)
            a = np.asarray(a).squeeze()
            if a.ndim == 0: a = np.array([a], dtype=np.float32)
            return a.astype(np.float32)
    except Exception:
        return None
    return None

def dim_of(p):
    v = load_vec(p)
    return None if v is None else int(v.shape[0])

# Diagnose dimensions
usable = df[(df["vision_feat_path"].notna()) | (df["audio_feat_path"].notna())].copy()
usable["v_dim"] = usable["vision_feat_path"].apply(dim_of)
usable["a_dim"] = usable["audio_feat_path"].apply(dim_of)

v_counts = usable["v_dim"].dropna().astype(int).value_counts().sort_index()
a_counts = usable["a_dim"].dropna().astype(int).value_counts().sort_index()
print("Vision dims:", dict(v_counts))
print("Audio  dims:", dict(a_counts))

# Strategy
USE_PADDING = False  # False = keep majority dim; True = pad to max dim
v_major = int(v_counts.idxmax()) if len(v_counts) else None
a_major = int(a_counts.idxmax()) if len(a_counts) else None
print("Majority dims -> vision:", v_major, "| audio:", a_major)

def pad_stack(vectors, target=None):
    vecs = [v for v in vectors if isinstance(v, np.ndarray)]
    if len(vecs)==0: return np.zeros((0,0), np.float32)
    if target is None:
        target = max(v.shape[0] for v in vecs)
    M = np.zeros((len(vecs), target), np.float32)
    for i, v in enumerate(vecs):
        d = min(target, v.shape[0])
        M[i,:d] = v[:d]
    return M

# --- Builders separated per setup to avoid None leakage ---

def build_V_only(df_in):
    V, Y, G = [], [], []
    for _, r in df_in.iterrows():
        v = load_vec(r["vision_feat_path"]) if isinstance(r["vision_feat_path"], str) else None
        if v is None: 
            continue
        if not USE_PADDING and v_major is not None and v.shape[0] != v_major:
            continue
        V.append(v)
        Y.append(label2id[r["label"]])
        G.append(str(r["video_id"]))
    if not V:
        return np.zeros((0,0), np.float32), np.zeros((0,), np.int64), np.array([])
    X = pad_stack(V, None if USE_PADDING else v_major)
    return X, np.array(Y, np.int64), np.array(G)

def build_A_only(df_in):
    A, Y, G = [], [], []
    for _, r in df_in.iterrows():
        a = load_vec(r["audio_feat_path"]) if isinstance(r["audio_feat_path"], str) else None
        if a is None: 
            continue
        if not USE_PADDING and a_major is not None and a.shape[0] != a_major:
            continue
        A.append(a)
        Y.append(label2id[r["label"]])
        G.append(str(r["video_id"]))
    if not A:
        return np.zeros((0,0), np.float32), np.zeros((0,), np.int64), np.array([])
    X = pad_stack(A, None if USE_PADDING else a_major)
    return X, np.array(Y, np.int64), np.array(G)

def build_VA_fusion(df_in):
    V, A, Y, G = [], [], [], []
    for _, r in df_in.iterrows():
        v = load_vec(r["vision_feat_path"]) if isinstance(r["vision_feat_path"], str) else None
        a = load_vec(r["audio_feat_path"])  if isinstance(r["audio_feat_path"],  str) else None
        # Fusion requires BOTH
        if v is None or a is None:
            continue
        if not USE_PADDING:
            if v_major is not None and v.shape[0] != v_major:
                continue
            if a_major is not None and a.shape[0] != a_major:
                continue
        V.append(v); A.append(a)
        Y.append(label2id[r["label"]])
        G.append(str(r["video_id"]))
    if not V:
        return np.zeros((0,0), np.float32), np.zeros((0,), np.int64), np.array([])
    V_mat = pad_stack(V, None if USE_PADDING else v_major)
    A_mat = pad_stack(A, None if USE_PADDING else a_major)
    X = np.concatenate([V_mat, A_mat], axis=1)
    return X, np.array(Y, np.int64), np.array(G)

# Label map (ensure exists)
labels_sorted = sorted(usable["label"].unique())
label2id = {lbl:i for i,lbl in enumerate(labels_sorted)}
id2label = {i:lbl for lbl,i in label2id.items()}
print("Classes:", label2id)

# Build three sets
Xv,  Yv,  Gv  = build_V_only(usable)
Xa,  Ya,  Ga  = build_A_only(usable)
Xva, Yva, Gva = build_VA_fusion(usable)

print("Vision-only:", Xv.shape,  "labels:", len(Yv))
print("Audio-only :", Xa.shape,  "labels:", len(Ya))
print("V+A fusion :", Xva.shape, "labels:", len(Yva))
if Xv.size==0 and Xa.size==0 and Xva.size==0:
    print("WARNING: No usable features found. Recheck file names and paths.")


Vision dims: {2048: 1081}
Audio  dims: {64: 1068}
Majority dims -> vision: 2048 | audio: 64
Classes: {'hateful': 0, 'normal': 1}
Vision-only: (1081, 2048) labels: 1081
Audio-only : (1068, 64) labels: 1068
V+A fusion : (1066, 2112) labels: 1066


In [12]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np
import pandas as pd

def eval_groupkfold(X, Y, G, name):
    if X.size == 0:
        print(f"{name}: EMPTY, skip.")
        return {"name":name, "LR_acc":np.nan, "LR_f1m":np.nan, "MLP_acc":np.nan, "MLP_f1m":np.nan}

    gkf = GroupKFold(n_splits=min(5, len(np.unique(G))))
    def run(clf):
        yt, yp = [], []
        for tr, te in gkf.split(X, Y, G):
            clf.fit(X[tr], Y[tr])
            yp.append(clf.predict(X[te])); yt.append(Y[te])
        yt = np.concatenate(yt); yp = np.concatenate(yp)
        acc = accuracy_score(yt, yp)
        f1m = f1_score(yt, yp, average="macro")
        rep = classification_report(yt, yp, target_names=[k for k,_ in sorted(label2id.items(), key=lambda kv: kv[1])], zero_division=0)
        return acc, f1m, rep

    lr  = LogisticRegression(penalty="l2", C=1.0, solver="saga", class_weight="balanced", max_iter=3000, n_jobs=-1)
    mlp = MLPClassifier(hidden_layer_sizes=(256,), activation="relu", alpha=1e-4,
                        batch_size=128, learning_rate_init=1e-3, max_iter=80, early_stopping=True, random_state=42)

    acc_lr, f1_lr, rep_lr     = run(lr)
    acc_mlp, f1_mlp, rep_mlp  = run(mlp)

    print(f"[{name}]  LR : acc={acc_lr:.3f}  f1m={f1_lr:.3f}")
    print(rep_lr)
    print(f"[{name}]  MLP: acc={acc_mlp:.3f} f1m={f1_mlp:.3f}")
    print(rep_mlp)

    return {"name":name, "LR_acc":acc_lr, "LR_f1m":f1_lr, "report_LR":rep_lr,
            "MLP_acc":acc_mlp, "MLP_f1m":f1_mlp, "report_MLP":rep_mlp}

rows = []
rows.append(eval_groupkfold(Xv,  Yv,  Gv,  "Vision-only"))
rows.append(eval_groupkfold(Xa,  Ya,  Ga,  "Audio-only"))
rows.append(eval_groupkfold(Xva, Yva, Gva, "V+A (Early Fusion)"))

summary = pd.DataFrame([{k:v for k,v in r.items() if isinstance(v,(int,float,str))} for r in rows]).sort_values("MLP_f1m", ascending=False)
OUT = PROC_DIR / "baseline_summary_VA.csv"
summary.to_csv(OUT, index=False)
print("Saved:", OUT)
summary

[Vision-only]  LR : acc=0.716  f1m=0.705
              precision    recall  f1-score   support

     hateful       0.64      0.66      0.65       429
      normal       0.77      0.75      0.76       652

    accuracy                           0.72      1081
   macro avg       0.70      0.71      0.71      1081
weighted avg       0.72      0.72      0.72      1081

[Vision-only]  MLP: acc=0.717 f1m=0.702
              precision    recall  f1-score   support

     hateful       0.65      0.62      0.64       429
      normal       0.76      0.78      0.77       652

    accuracy                           0.72      1081
   macro avg       0.70      0.70      0.70      1081
weighted avg       0.72      0.72      0.72      1081



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[Audio-only]  LR : acc=0.632  f1m=0.626
              precision    recall  f1-score   support

     hateful       0.54      0.63      0.58       429
      normal       0.72      0.64      0.67       639

    accuracy                           0.63      1068
   macro avg       0.63      0.63      0.63      1068
weighted avg       0.64      0.63      0.64      1068

[Audio-only]  MLP: acc=0.576 f1m=0.467
              precision    recall  f1-score   support

     hateful       0.42      0.15      0.23       429
      normal       0.60      0.86      0.71       639

    accuracy                           0.58      1068
   macro avg       0.51      0.51      0.47      1068
weighted avg       0.53      0.58      0.51      1068



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[V+A (Early Fusion)]  LR : acc=0.735  f1m=0.726
              precision    recall  f1-score   support

     hateful       0.66      0.70      0.68       427
      normal       0.79      0.76      0.77       639

    accuracy                           0.73      1066
   macro avg       0.72      0.73      0.73      1066
weighted avg       0.74      0.73      0.74      1066

[V+A (Early Fusion)]  MLP: acc=0.707 f1m=0.681
              precision    recall  f1-score   support

     hateful       0.67      0.53      0.59       427
      normal       0.72      0.83      0.77       639

    accuracy                           0.71      1066
   macro avg       0.70      0.68      0.68      1066
weighted avg       0.70      0.71      0.70      1066

Saved: /kaggle/working/baseline_summary_VA.csv


,name,LR_acc,LR_f1m,report_LR,MLP_acc,MLP_f1m,report_MLP
0,Vision-only,0.716004,0.705085,precision recall f1-score ...,0.716929,0.702123,precision recall f1-score ...
2,V+A (Early Fusion),0.734522,0.726460,precision recall f1-score ...,0.707317,0.681407,precision recall f1-score ...
1,Audio-only,0.632022,0.625866,precision recall f1-score ...,0.575843,0.466786,precision recall f1-score ...


In [13]:
from itertools import islice

print("Sample feature files in vision/:")
for p in islice(sorted(V_DIR.glob("*.npy")), 5):
    print(" -", p.name)

print("\nSample feature files in audio/:")
for p in islice(sorted(A_DIR.glob("*.npy")), 5):
    print(" -", p.name)

print("\nSample rows from manifest (paths normalized):")
df[["sample_id","video_id","label","vision_feat_path","audio_feat_path"]].head(5)


Sample feature files in vision/:
 - HateMM__hate_video_1.mp4__0_end__hateful.npy
 - HateMM__hate_video_10.mp4__0_end__hateful.npy
 - HateMM__hate_video_100.mp4__0_end__hateful.npy
 - HateMM__hate_video_101.mp4__0_end__hateful.npy
 - HateMM__hate_video_102.mp4__0_end__hateful.npy

Sample feature files in audio/:
 - HateMM__hate_video_1.mp4__0_end__hateful.npy
 - HateMM__hate_video_10.mp4__0_end__hateful.npy
 - HateMM__hate_video_100.mp4__0_end__hateful.npy
 - HateMM__hate_video_101.mp4__0_end__hateful.npy
 - HateMM__hate_video_102.mp4__0_end__hateful.npy

Sample rows from manifest (paths normalized):


,sample_id,video_id,label,vision_feat_path,audio_feat_path
0,HateMM__hate_video_1.mp4__0_end__hateful,hate_video_1.mp4,hateful,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
1,HateMM__hate_video_2.mp4__0_end__hateful,hate_video_2.mp4,hateful,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
2,HateMM__non_hate_video_1.mp4__0_end__normal,non_hate_video_1.mp4,normal,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
3,HateMM__hate_video_3.mp4__0_end__hateful,hate_video_3.mp4,hateful,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...
4,HateMM__non_hate_video_2.mp4__0_end__normal,non_hate_video_2.mp4,normal,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...


In [14]:
from pathlib import Path
import pandas as pd
import numpy as np

# === đường dẫn dữ liệu đã chuẩn hoá (từ notebook preprocess) ===
PROC_DIR    = Path("/kaggle/working")
MANIFEST_IN = PROC_DIR / "hatemm_train_manifest.rewritten.csv"  # đã rewrite sang CACHE_DIR/input

assert MANIFEST_IN.exists(), "Thiếu manifest. Hãy chạy notebook preprocessing + rewrite trước."

# === nơi đọc feature đã cache (đã upload sang /kaggle/input/<dataset>) ===
# VD: /kaggle/input/thesis-data-processed chứa 2 thư mục vision/ và audio/
CACHE_DIR = Path("/kaggle/input/thesis-data-processed/features")   # <-- đổi theo tên dataset input của bạn
V_DIR = CACHE_DIR / "vision"
A_DIR = CACHE_DIR / "audio"
T_DIR = CACHE_DIR / "text"    # sẽ sinh mới (embedding .npy) nhưng vì /kaggle/input read-only -> lưu sang working

# === nơi ghi output mới (có quyền ghi) ===
WORK_CACHE = Path("/kaggle/working/cache_hatemm_plus")
V_DIR_WORK = WORK_CACHE / "vision" ; V_DIR_WORK.mkdir(parents=True, exist_ok=True)
A_DIR_WORK = WORK_CACHE / "audio"  ; A_DIR_WORK.mkdir(parents=True, exist_ok=True)
T_DIR_WORK = WORK_CACHE / "text"   ; T_DIR_WORK.mkdir(parents=True, exist_ok=True)

# manifest
df = pd.read_csv(MANIFEST_IN)
print("Loaded rows:", len(df))
df.head(2)


Loaded rows: 1083


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,dataset,video_id,video_path,start_sec,end_sec,target_group,label,binary_offensive,sample_id,video_seconds,segment_seconds,is_broken,vision_feat_path,audio_feat_path,has_vision,has_audio
0,HateMM,hate_video_1.mp4,/kaggle/input/thesis-dataset/Thesis_dataset/ha...,NaN,NaN,['Blacks'],hateful,1,HateMM__hate_video_1.mp4__0_end__hateful,94.998,NaN,False,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...,True,True
1,HateMM,hate_video_2.mp4,/kaggle/input/thesis-dataset/Thesis_dataset/ha...,NaN,NaN,['Blacks'],hateful,1,HateMM__hate_video_2.mp4__0_end__hateful,129.160,NaN,False,/kaggle/input/thesis-data-processed/features/v...,/kaggle/input/thesis-data-processed/features/a...,True,True


In [15]:
# Giữ nguyên file ở /kaggle/input, chỉ "viết lại path" trỏ tới đúng nơi đọc
# Để thống nhất, chúng ta cho vision/audio path dùng trực tiếp từ INPUT; text sẽ nằm ở WORKING.
def fix_col_path(series, target_root):
    out = []
    for p in series.fillna(""):
        base = Path(p).name if isinstance(p, str) and p else None
        if base:
            out.append(str(target_root / base))
        else:
            out.append(np.nan)
    return out

df["vision_feat_path"] = fix_col_path(df.get("vision_feat_path", pd.Series([np.nan]*len(df))), V_DIR)
df["audio_feat_path"]  = fix_col_path(df.get("audio_feat_path",  pd.Series([np.nan]*len(df))), A_DIR)

# Đảm bảo sample_id tồn tại (dự phòng)
if "sample_id" not in df.columns:
    def make_sid(r):
        s = "0" if pd.isna(r.get("start_sec")) else f"{float(r['start_sec']):.3f}"
        e = "end" if pd.isna(r.get("end_sec")) else f"{float(r['end_sec']):.3f}"
        return f"{r.get('dataset','HateMM')}__{r['video_id']}__{s}_{e}__{r['label']}"
    df["sample_id"] = df.apply(make_sid, axis=1)

print("vision non-null ratio:", df["vision_feat_path"].notna().mean())
print("audio  non-null ratio:", df["audio_feat_path"].notna().mean())

vision non-null ratio: 1.0
audio  non-null ratio: 1.0


In [16]:
import subprocess, shlex, pandas as pd, numpy as np
import whisper, torch
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"
WHISPER_MODEL = "small"  # tiny/base/small; nếu hạn chế compute -> dùng "tiny" hoặc "base"
asr = whisper.load_model(WHISPER_MODEL, device=device)

# Ta cần file video gốc để cắt theo (start_sec, end_sec).
# Nếu manifest có cột 'video_path' -> dùng trực tiếp.
# Nếu không, bạn phải thêm cột đó trước (mapping từ video_id -> đường dẫn thực tế).
assert "video_path" in df.columns, "Thiếu 'video_path' trong manifest. Cần cột này để ASR."

WAV_TMP = Path("/kaggle/working/asr_wav")
WAV_TMP.mkdir(parents=True, exist_ok=True)

def ffmpeg_extract_wav(video_path: str, out_wav: Path, start=None, end=None, sr=16000):
    ss = f"-ss {float(start):.3f}" if start is not None and not pd.isna(start) else ""
    to = f"-to {float(end):.3f}"   if end is not None and not pd.isna(end)   else ""
    cmd = f'ffmpeg -hide_banner -loglevel error -y {ss} -i {shlex.quote(video_path)} {to} -vn -ac 1 -ar {sr} {shlex.quote(str(out_wav))}'
    ok = subprocess.run(cmd, shell=True).returncode == 0
    return str(out_wav) if ok and out_wav.exists() and out_wav.stat().st_size>0 else None

def asr_one(row):
    sid = row["sample_id"]
    s, e = row.get("start_sec"), row.get("end_sec")
    vpath = row["video_path"]
    wavp = WAV_TMP / f"{sid}.wav"
    wav_file = ffmpeg_extract_wav(vpath, wavp, s, e, sr=16000)
    if wav_file is None: return None
    # whisper transcribe
    try:
        out = asr.transcribe(wav_file, language="en")  # nếu video không phải tiếng Anh, bỏ language để auto
        text = (out.get("text") or "").strip()
        return text
    except Exception:
        return None
    finally:
        try: wavp.unlink()
        except: pass

# ASR (có thể tốn thời gian; chạy theo batches/limit nếu cần)
if "transcript" not in df.columns:
    df["transcript"] = None

todo_idx = df[df["transcript"].isna()].index
print("ASR pending:", len(todo_idx))
for i in todo_idx:
    t = asr_one(df.loc[i])
    df.at[i, "transcript"] = t
    if (i % 50) == 0:
        print(f"ASR progress i={i}, got={t[:60] if t else None}")

# Lưu lại intermediate transcript
TRANS_OUT = PROC_DIR / "hatemm_transcripts.csv"
df[["sample_id","video_id","label","transcript"]].to_csv(TRANS_OUT, index=False)
print("Saved transcripts:", TRANS_OUT)

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 79.9MiB/s]


ASR pending: 1083
ASR progress i=0, got=You You You You


Output file #0 does not contain any stream
Output file #0 does not contain any stream
Output file #0 does not contain any stream


ASR progress i=50, got=Cancer nigger, how does it feel to be dying of a Jewish dise
ASR progress i=100, got=Now, how do I know that the white people know that we are go


Output file #0 does not contain any stream


...

ASR progress i=750, got=The football team coming back in 2011. So you know, he likes
ASR progress i=800, got=How can we ever be expected to be expected as a people, as a
ASR progress i=850, got=Trump supporters just took my job away from me. I have gotte
ASR progress i=900, got=Before I was even born, my father was in the Congress of the


Output file #0 does not contain any stream


ASR progress i=950, got=Here's Dan, riding in the fire, riding in crosses by the nig


Output file #0 does not contain any stream


ASR progress i=1000, got=Well boys, as far as we go... This will allow space sensors 
ASR progress i=1050, got=紅藏兒隊! . . . . . . . . . , . . . . . . . . .aines de No, God 
Saved transcripts: /kaggle/working/hatemm_transcripts.csv


In [17]:
import torch, numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# BERT-base-uncased. Nếu không có internet, hãy Add Data một dataset chứa model này,
# sau đó đổi model_name thành đường dẫn local (e.g., "/kaggle/input/bert-base-uncased").
model_name = "bert-base-uncased"
tok = AutoTokenizer.from_pretrained(model_name)
bert = AutoModel.from_pretrained(model_name).to(device)
bert.eval()

def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).type(last_hidden_state.dtype)
    masked = last_hidden_state * mask
    return masked.sum(dim=1) / mask.sum(dim=1).clamp(min=1e-6)

text_paths = []
for i, r in tqdm(df.iterrows(), total=len(df)):
    sid = r["sample_id"]
    txt = str(r.get("transcript") or "").strip()
    if not txt:
        text_paths.append(np.nan)
        continue
    out_path = T_DIR_WORK / f"{sid}.npy"
    if out_path.exists():
        text_paths.append(str(out_path)); continue
    try:
        enc = tok(txt, truncation=True, max_length=256, padding=True, return_tensors="pt")
        enc = {k:v.to(device) for k,v in enc.items()}
        with torch.no_grad():
            out = bert(**enc).last_hidden_state  # (1, L, H)
            emb = mean_pool(out, enc["attention_mask"]).squeeze(0).detach().cpu().numpy().astype(np.float32)  # (H,)
        np.save(out_path, emb)
        text_paths.append(str(out_path))
    except Exception:
        text_paths.append(np.nan)

df["text_feat_path"] = text_paths
print("text non-null ratio:", df["text_feat_path"].notna().mean())

# Lưu manifest mới có text
MANIFEST_TXT = PROC_DIR / "hatemm_train_manifest.with_text.csv"
df.to_csv(MANIFEST_TXT, index=False)
print("Saved:", MANIFEST_TXT)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 1083/1083 [00:16<00:00, 65.13it/s]

text non-null ratio: 0.971375807940905
Saved: /kaggle/working/hatemm_train_manifest.with_text.csv


In [18]:
import numpy as np
import pandas as pd
from pathlib import Path

def load_vec(p):
    try:
        p = Path(p)
        if p.exists():
            a = np.load(p, mmap_mode=None)
            return a.squeeze().astype(np.float32)
    except Exception:
        return None
    return None

def dim_of(p):
    try:
        a = load_vec(p)
        return None if a is None else a.shape[0]
    except: return None

use = df.copy()
use["v_dim"] = use["vision_feat_path"].apply(dim_of)
use["a_dim"] = use["audio_feat_path"].apply(dim_of)
use["t_dim"] = use["text_feat_path"].apply(dim_of)

v_counts = use["v_dim"].dropna().astype(int).value_counts()
a_counts = use["a_dim"].dropna().astype(int).value_counts()
t_counts = use["t_dim"].dropna().astype(int).value_counts()
print("V dim:", v_counts.to_dict())
print("A dim:", a_counts.to_dict())
print("T dim:", t_counts.to_dict())

USE_PADDING = False  # True: pad mỗi modality đến max; False: chỉ giữ majority-dim cho từng modality

v_major = int(v_counts.idxmax()) if len(v_counts) else None
a_major = int(a_counts.idxmax()) if len(a_counts) else None
t_major = int(t_counts.idxmax()) if len(t_counts) else None
print("major dims ->", v_major, a_major, t_major)

labels_sorted = sorted(use["label"].unique())
label2id = {lbl:i for i,lbl in enumerate(labels_sorted)}

def pad_stack(vectors, target=None):
    if len(vectors)==0: return np.zeros((0,0), np.float32)
    if target is None: target = max(v.shape[0] for v in vectors)
    M = np.zeros((len(vectors), target), np.float32)
    for i, v in enumerate(vectors):
        d = min(target, v.shape[0])
        M[i,:d] = v[:d]
    return M

def collect(mods=("v","a","t")):
    rows = []
    for _, r in use.iterrows():
        feats = []
        dims_ok = True
        if "v" in mods:
            v = load_vec(r["vision_feat_path"])
            if v is None: dims_ok=False
            elif not USE_PADDING and v.shape[0]!=v_major: dims_ok=False
            else: feats.append(("v", v))
        if "a" in mods:
            a = load_vec(r["audio_feat_path"])
            if a is None: dims_ok=False
            elif not USE_PADDING and a.shape[0]!=a_major: dims_ok=False
            else: feats.append(("a", a))
        if "t" in mods:
            t = load_vec(r["text_feat_path"])
            if t is None: dims_ok=False
            elif not USE_PADDING and t.shape[0]!=t_major: dims_ok=False
            else: feats.append(("t", t))
        if not dims_ok or not feats: continue
        rows.append((feats, label2id[r["label"]], str(r["video_id"])))
    if not rows:
        return np.zeros((0,0), np.float32), np.zeros((0,), np.int64), np.array([])
    Xs = {"v":[], "a":[], "t":[]}
    Y, G = [], []
    for feats, y, g in rows:
        for k, vec in feats:
            Xs[k].append(vec)
        Y.append(y); G.append(g)
    # pad per-modality then concat in fixed order
    mats = []
    for k in mods:
        if len(Xs[k])==0: continue
        if USE_PADDING:
            mats.append(pad_stack(Xs[k]))
        else:
            target = {"v":v_major, "a":a_major, "t":t_major}[k]
            mats.append(pad_stack(Xs[k], target))
    X = np.concatenate(mats, axis=1) if len(mats)>1 else mats[0]
    Y = np.array(Y, np.int64); G = np.array(G)
    return X, Y, G

# Build all combos
Xv,  Yv,  Gv  = collect(("v",))
Xa,  Ya,  Ga  = collect(("a",))
Xt,  Yt,  Gt  = collect(("t",))
Xva, Yva, Gva = collect(("v","a"))
Xvt, Yvt, Gvt = collect(("v","t"))
Xat, Yat, Gat = collect(("a","t"))
Xvat, Yvat, Gvat = collect(("v","a","t"))

print("Shapes:")
for name, X, Y in [
    ("V", Xv, Yv), ("A", Xa, Ya), ("T", Xt, Yt),
    ("V+A", Xva, Yva), ("V+T", Xvt, Yvt), ("A+T", Xat, Yat),
    ("V+A+T", Xvat, Yvat)
]:
    print(f"{name:6s}: {X.shape}  labels={len(Y)}")

V dim: {2048: 1081}
A dim: {64: 1068}
T dim: {768: 1052}
major dims -> 2048 64 768
Shapes:
V     : (1081, 2048)  labels=1081
A     : (1068, 64)  labels=1068
T     : (1052, 768)  labels=1052
V+A   : (1066, 2112)  labels=1066
V+T   : (1050, 2816)  labels=1050
A+T   : (1052, 832)  labels=1052
V+A+T : (1050, 2880)  labels=1050


In [19]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np, pandas as pd

def eval_block(X, Y, G, name):
    if X.size==0:
        return {"Model":name, "LR_acc":np.nan, "LR_f1":np.nan, "MLP_acc":np.nan, "MLP_f1":np.nan}
    gkf = GroupKFold(n_splits=min(5, len(np.unique(G))))
    def go(clf):
        yt, yp = [], []
        for tr, te in gkf.split(X, Y, G):
            clf.fit(X[tr], Y[tr])
            yp.append(clf.predict(X[te])); yt.append(Y[te])
        yt = np.concatenate(yt); yp = np.concatenate(yp)
        return accuracy_score(yt, yp), f1_score(yt, yp, average="macro")
    lr  = LogisticRegression(penalty="l2", C=1.0, solver="saga", class_weight="balanced", max_iter=3000, n_jobs=-1)
    mlp = MLPClassifier(hidden_layer_sizes=(256,), activation="relu", alpha=1e-4,
                        batch_size=128, learning_rate_init=1e-3, max_iter=80, early_stopping=True, random_state=42)
    acc_lr, f1_lr = go(lr)
    acc_mlp, f1_mlp = go(mlp)
    print(f"{name:10s} | LR  acc={acc_lr:.3f} f1={f1_lr:.3f} | MLP acc={acc_mlp:.3f} f1={f1_mlp:.3f}")
    return {"Model":name, "LR_acc":acc_lr, "LR_f1":f1_lr, "MLP_acc":acc_mlp, "MLP_f1":f1_mlp}

rows = []
rows.append(eval_block(Xv, Yv, Gv, "V"))
rows.append(eval_block(Xa, Ya, Ga, "A"))
rows.append(eval_block(Xt, Yt, Gt, "T"))
rows.append(eval_block(Xva, Yva, Gva, "V+A"))
rows.append(eval_block(Xvt, Yvt, Gvt, "V+T"))
rows.append(eval_block(Xat, Yat, Gat, "A+T"))
rows.append(eval_block(Xvat, Yvat, Gvat, "V+A+T"))

res_df = pd.DataFrame(rows).sort_values("MLP_f1", ascending=False)
OUT_SUM = PROC_DIR / "baseline_summary_VAT.csv"
res_df.to_csv(OUT_SUM, index=False)
print("Saved:", OUT_SUM)
res_df

V          | LR  acc=0.716 f1=0.705 | MLP acc=0.717 f1=0.702


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


A          | LR  acc=0.632 f1=0.626 | MLP acc=0.576 f1=0.467
T          | LR  acc=0.760 f1=0.752 | MLP acc=0.775 f1=0.763


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


V+A        | LR  acc=0.735 f1=0.726 | MLP acc=0.707 f1=0.681
V+T        | LR  acc=0.786 f1=0.777 | MLP acc=0.801 f1=0.792


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


A+T        | LR  acc=0.779 f1=0.773 | MLP acc=0.776 f1=0.771


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


V+A+T      | LR  acc=0.797 f1=0.791 | MLP acc=0.798 f1=0.786
Saved: /kaggle/working/baseline_summary_VAT.csv


,Model,LR_acc,LR_f1,MLP_acc,MLP_f1
4,V+T,0.785714,0.776697,0.800952,0.792410
6,V+A+T,0.797143,0.791347,0.798095,0.785755
5,A+T,0.778517,0.772506,0.775665,0.770621
2,T,0.759506,0.752169,0.774715,0.762883
0,V,0.716004,0.705085,0.716929,0.702123
3,V+A,0.734522,0.726460,0.707317,0.681407
1,A,0.632022,0.625866,0.575843,0.466786


In [20]:
import torch, torch.nn as nn
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, f1_score

device = "cuda" if torch.cuda.is_available() else "cpu"

class AttnFusion(nn.Module):
    """
    Nhận tối đa 3 token (V, A, T) với chiều d_v, d_a, d_t.
    Project -> d_model, +type embedding, TransformerEncoder, mean-pool, classifier.
    """
    def __init__(self, dims, n_classes, d_model=256, nhead=8, nlayers=2, dropout=0.1):
        super().__init__()
        self.proj = nn.ModuleList([nn.Linear(d, d_model) for d in dims])  # one per modality
        self.type_embed = nn.Embedding(len(dims), d_model)
        enc_lyr = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=d_model*4, dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(enc_lyr, num_layers=nlayers)
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU(), nn.Dropout(dropout), nn.Linear(d_model, n_classes))

    def forward(self, xs):  # list of (B, d_k)
        B = xs[0].size(0)
        tokens = []
        for i, x in enumerate(xs):
            tokens.append(self.proj[i](x) + self.type_embed.weight[i])  # (B, d_model)
        x = torch.stack(tokens, dim=1)  # (B, K, d_model)
        x = self.encoder(x)
        x = self.norm(x.mean(dim=1))
        return self.head(x)

def attn_gkf(Xs, Y, G, epochs=12, lr=1e-3, batch=128):
    gkf = GroupKFold(n_splits=min(5, len(np.unique(G))))
    all_true, all_pred = [], []
    for tr, te in gkf.split(Xs[0], Y, G):
        # build tensors
        Xtr = [torch.from_numpy(x[tr]).to(device) for x in Xs]
        Xte = [torch.from_numpy(x[te]).to(device) for x in Xs]
        ytr = torch.from_numpy(Y[tr]).to(device)
        yte = Y[te]

        model = AttnFusion([x.shape[1] for x in Xtr], n_classes=len(np.unique(Y))).to(device)
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
        crit = nn.CrossEntropyLoss()

        model.train()
        for ep in range(epochs):
            idx = np.random.permutation(len(tr))
            for i in range(0, len(tr), batch):
                sel = idx[i:i+batch]
                batch_x = [x[sel] for x in Xtr]
                batch_y = ytr[sel]
                opt.zero_grad()
                logits = model(batch_x)
                loss = crit(logits, batch_y)
                loss.backward()
                opt.step()

        model.eval()
        with torch.no_grad():
            logits = model(Xte).cpu().numpy()
        pred = logits.argmax(axis=1)
        all_true.append(yte); all_pred.append(pred)

    all_true = np.concatenate(all_true); all_pred = np.concatenate(all_pred)
    acc = accuracy_score(all_true, all_pred)
    f1m = f1_score(all_true, all_pred, average="macro")
    return acc, f1m

# Chạy cho V+A+T nếu khả dụng, nếu không thì chạy V+A
if Xvat.size:
    dims = [Xv.shape[1], Xa.shape[1], Xt.shape[1]]
    # Cần đảm bảo những ma trận này align hàng — nếu bạn build bằng collect(("v","a","t")) thì đã align.
    Xs = []
    # tái build đúng hàng cho V,A,T dựa trên collect(("v","a","t")):
    # ở đây Xvat là concat [V|A|T], nên tách ra:
    dv, da, dt = dims
    Xs = [Xvat[:, :dv], Xvat[:, dv:dv+da], Xvat[:, dv+da:dv+da+dt]]
    acc, f1m = attn_gkf(Xs, Yvat, Gvat, epochs=12, lr=1e-3, batch=128)
    print(f"[Attn V+A+T] acc={acc:.3f} f1={f1m:.3f}")
elif Xva.size:
    dv, da = Xv.shape[1], Xa.shape[1]
    Xs = [Xva[:, :dv], Xva[:, dv:dv+da]]
    acc, f1m = attn_gkf(Xs, Yva, Gva, epochs=12, lr=1e-3, batch=128)
    print(f"[Attn V+A] acc={acc:.3f} f1={f1m:.3f}")
else:
    print("No multimodal matrices available for attention fusion.")

[Attn V+A+T] acc=0.599 f1=0.484
